In [1]:
# Auto Reload
%load_ext autoreload
%autoreload 2

In [2]:
# Import Modules
import numpy as np
import copy
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup
from pprint import pprint

In [3]:
# Import Custom Modules
from webscrape import *
from helpers import *

In [4]:
# Create Dictionaries by Year with Empty DataFrame as Vaule
top_charts = {
    2015: pd.DataFrame(),
    2016: pd.DataFrame(),
    2017: pd.DataFrame(),
    2018: pd.DataFrame(),
    2019: pd.DataFrame(),
}

In [ ]:
# 2019 Info
top_songs_2019 = get_top_songs(2019)
top_2019 = [get_full_lyrics(*x) for x in top_songs_2019]

In [ ]:
# 2018 Info
top_songs_2018 = get_top_songs(2018)
top_2018 = [get_full_lyrics(*x) for x in top_songs_2018]

In [ ]:
# Populate DataFrames by Year

In [ ]:
top_2019